In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv


# I using text to text generation to complete this project

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv')

In [4]:
data.head(10)

,Unnamed: 0,question,answer
0,0,"hi, how are you doing?",i'm fine. how about yourself?
1,1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,3,no problem. so how have you been?,i've been great. what about you?
4,4,i've been great. what about you?,i've been good. i'm in school right now.
5,5,i've been good. i'm in school right now.,what school do you go to?
6,6,what school do you go to?,i go to pcc.
7,7,i go to pcc.,do you like it there?
8,8,do you like it there?,it's okay. it's a really big campus.
9,9,it's okay. it's a really big campus.,good luck with school.


In [5]:
data.tail(10)

,Unnamed: 0,question,answer
3715,3715,what's wrong with it?,it aches most of the time.
3716,3716,it aches most of the time.,what do you think it is?
3717,3717,what do you think it is?,i don't know. i think it's old age.
3718,3718,i don't know. i think it's old age.,"if it's old age, why don't both of your hands ..."
3719,3719,"if it's old age, why don't both of your hands ...",that's a good question. maybe it's not old age.
3720,3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,3721,are you right-handed?,yes. all my life.
3722,3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.
3724,3724,but i do all my writing with my right hand.,start typing instead. that way your left hand ...


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3725 entries, 0 to 3724
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3725 non-null   int64 
 1   question    3725 non-null   object
 2   answer      3725 non-null   object
dtypes: int64(1), object(2)
memory usage: 87.4+ KB


In [7]:
# delete column unnamed

del data['Unnamed: 0']

In [8]:
data.head(10)

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.
5,i've been good. i'm in school right now.,what school do you go to?
6,what school do you go to?,i go to pcc.
7,i go to pcc.,do you like it there?
8,do you like it there?,it's okay. it's a really big campus.
9,it's okay. it's a really big campus.,good luck with school.


# Create text to text generation using model google/flan-t5-base

# Q1 : hi, how are you doing?

In [1]:
!pip install huggingface


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install happytransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00

[notice] A new release o

In [2]:
!pip install torch transformers tqdm

/usr/local/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "hi, how are you doing?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Original Question: hi, how are you doing?
Paraphrases:
1. How are you feeling?
2. Hi there, how are you?
3. What's up, how are you?
4. Can you tell me how things are going?
5. How is everything going on?


# Q2 : i'm fine. how about yourself?

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "i'm fine. how about yourself?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: i'm fine. how about yourself?
Paraphrases:
1. I'm doing well. How are you?
2. Hi I'm doing fine. How are you?
3. "It's all good. How are you?"
4. How do you feel about your relationship?
5. Is it safe to say that I am in good physical condition?


# Q3 : no problem. so how have you been?

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "no problem. so how have you been?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: no problem. so how have you been?
Paraphrases:
1. Can you tell me how your day has been?
2. I'm sorry, could you tell me how your day has been?
3. Hi, how's it going?
4. How are you doing?
5. What's your current situation?


# Q4 : i've been good. i'm in school right now.

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "i've been good. i'm in school right now."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: i've been good. i'm in school right now.
Paraphrases:
1. My academic performance is impressive, and I'm currently in the same class.
2. Currently, I'm in school and have been doing well.
3. It seems like my current situation isn't bad since I'm currently in school.
4. I've been doing a good job while attending school.
5. I'm doing well while I attend school right now.


# Q5 : what school do you go to?

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "what school do you go to?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: what school do you go to?
Paraphrases:
1. Which educational institution do you attend?
2. What is the name of your school?
3. What's the name of the school you attend?
4. To which educational institution do you attend?
5. Where do you go to school?


# Q6 : do you like it there?

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "do you like it there?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: do you like it there?
Paraphrases:
1. Is there something that appeals to you?
2. Do you find it enjoyable to be there?
3. Are you fond of it there?
4. Does it appeal to you in that place?
5. Would that be a suitable setting for you?


# Q7 : i've actually been pretty good. you?

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "i've actually been pretty good. you?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: i've actually been pretty good. you?
Paraphrases:
1. I'm not complaining, are you okay?
2. Have you found myself relatively well-suited to my interests and hobbies?
3. Do you think I'm pretty good?
4. Can it be said that I'm pretty good at something?
5. Would you say that I'm pretty good at it?


# Q8 : so how have you been lately?

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "so how have you been lately?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: so how have you been lately?
Paraphrases:
1. Can you tell me where you are?
2. How are you doing?
3. What's your current state?
4. How is everything going on with you lately?
5. Could you share with me your current whereabouts?


# Q9 : never better, thanks.

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "never better, thanks."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: never better, thanks.
Paraphrases:
1. Never in your dreams, thank you.
2. I'll never experience anything like it, thanks.
3. Thank you for the opportunity to witness never before.
4. It has never been better, thank you.
5. Definitely never more perfect.


# Q10 : which school do you attend?

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "which school do you attend?"
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: which school do you attend?
Paraphrases:
1. What is the school you attend?
2. Can you tell me which school you are currently studying?
3. Which educational institution do you belong to?
4. What's the name of the school you attend?
5. Where do you go to school?
